# WIF Serialization

WIF is the serialization of a Private Key.

In [1]:
from ecc import G
from hashlib import sha256 as s256
from hashlib import sha512 as s512
from ecc import PrivateKey

s = input("What is your code? ")
hashes = int(input("How many hashes?"))

for i in range(hashes):
    s = s512(s.encode()).hexdigest()
    t = s256(s.encode()).hexdigest()
s = (int(s, 16))
t = (int(t, 16))
secret = s*t

# use your phone number if you can't think of anything
#secret = 0x977090903897cfbb8e30a806ad5d0c2086cb42d3e341ba0efdccc31b4b278823058aa1c57ddd23fccf54eea8d71199b053ccac8c53c87b9dfafda9e32562f5d32

print("PK HEX: "+ hex(secret))
print('PK WIF (UC): ' + PrivateKey.wif(compressed=False, testnet=False))
print('PK WIF (C): ' + PrivateKey.wif(compressed=True, testnet=False))
# get the public point
point = secret*G
# if you completed the `address` method, just do the .address(testnet=True) method on the public point
print('Compressed: ' + (point.address(compressed=True, testnet=False)))
print('Unompressed: ' + (point.address(compressed=False, testnet=False)))

What is your code? bitcoin
How many hashes?43
PK HEX: 0x159c9cc78ae6d75a15694a735993d96b3dbeb2204a2b40f86afa251853bf9e0c84aa1b6b706239021cdc0e98e63ad291acd451d3ae065e4e49dc974fb9662e5b637b750b98fc311df68446a4adf862bc583ef90531b62976b86de68c4df4065


AttributeError: type object 'PrivateKey' has no attribute 'wif'

### Try it

#### Find the WIF Format of the following:

* \\(2^{256}-2^{199}\\), mainnet, compressed
* \\(2^{256}-2^{201}\\), testnet, uncompressed
* 0dba685b4511dbd3d368e5c4358a1277de9486447af7b3604a69b8d9d8b7889d, mainnet, uncompressed
* 1cca23de92fd1862fb5b76e5f4f50eb082165e5191e116c18ed1a6b24be6a53f, testnet, compressed

In [1]:
# Exercise 3.1
from helper import encode_base58_checksum

components = (
    # (secret, testnet, compressed)
    (2**256-2**199, False, True),
    (2**256-2**201, True, False),
    (0x0dba685b4511dbd3d368e5c4358a1277de9486447af7b3604a69b8d9d8b7889d, False, False),
    (0x1cca23de92fd1862fb5b76e5f4f50eb082165e5191e116c18ed1a6b24be6a53f, True, True),
)

# iterate through components
for secret, testnet, compressed in components:
    # get the private key in 32-byte big-endian: num.to_bytes(32, 'big')
    secret_bytes = secret.to_bytes(32, 'big')
    # prepend b'\x80' for mainnet, b'\xef' for testnet
    if testnet:
        prefix = b'\xef'
    else:
        prefix = b'\x80'
    # append b'\x01' for compressed
    if compressed:
        suffix = b'\x01'
    else:
        suffix = b''
    # base58 the whole thing with checksum
    wif = encode_base58_checksum(prefix + secret_bytes + suffix)
    print(wif)

L5oLkpV3aqBJ4BgssVAsax1iRa77G5CVYnv9adQ6Z87te7TyUdSC
93XfLeifX7Jx7n7ELGMAf1SUR6f9kgQs8Xke8WStMwUtrDucMzn
5HvLFPDVgFZRK9cd4C5jcWki5Skz6fmKqi1GQJf5ZoMofid2Dty
cNYfWuhDpbNM1JWc3c6JTrtrFVxU4AGhUKgw5f93NP2QaBqmxKkg


### Test Driven Exercise

In [10]:
from ecc import PrivateKey

class PrivateKey(PrivateKey):

    def wif(self, compressed=True, testnet=False):
        # convert the secret from integer to a 32-bytes in big endian using num.to_bytes(32, 'big')
        secret_bytes = self.secret.to_bytes(32, 'big')
        # prepend b'\xef' on testnet, b'\x80' on mainnet
        if testnet:
            prefix = b'\xef'
        else:
            prefix = b'\x80'
        # append b'\x01' if compressed
        if compressed:
            suffix = b'\x01'
        else:
            suffix = b''
        # encode_base58_checksum the whole thing
        return encode_base58_checksum(prefix + secret_bytes + suffix)